In [1]:
import docling

In [ ]:
from docling.document_converter import DocumentConverter

# source = "https://arxiv.org/pdf/2408.09869"  # document per local path or URL
source = "/app/app/data/1.4.9.Architecture des messages mécanismes de sécurité et protocole d’échanges_FM-TZ-L4-TTP-G2-LIV-SIC-00118-v0.4.2_Inprogress.docx"
converter = DocumentConverter()
result = converter.convert(source)
print(result.document.export_to_markdown()) 

<!-- image -->

**FO1.4.9 Architecture des messages, mécanismes de sécurité et protocole d’échanges**

Version : 0.4.2

Date de version : 11/05/2025

| Projet   | Zone   | Lot   | Phase   | Générations   | Type de document   | Emetteur   |   Numéro Chrono |   Version |   Indice |
|----------|--------|-------|---------|---------------|--------------------|------------|-----------------|-----------|----------|
| FM       | TZ     | L4    | TTP     | G2            | LIV                | SIC        |           00118 |         0 |      4.2 |

## 1 Informations du document

Périmètre de diffusion : Interne

Type : Initial

Date prévue de livraison : xxx

Statut : en cours

**Auteurs :**

| Pilote(s)           | Organisation   | Rôle dans le projet   |
|---------------------|----------------|-----------------------|
|                     |                |                       |
|                     |                |                       |
|                     |                |         

**Local**                                                                                                       **Cloud**                                                                                                                                                                                                 |
| **1. Ingestion**            | Tu téléverses ton `.docx` dans Streamlit, qui écrit le fichier sur disque local.                                                                                                                                                             | Tu envoies ton `.docx` vers un endpoint API (via une requête HTTPS) ou tu le stockes dans un bucket (S3, GCS).                                                                                                     |
| **2. Chunking**             | Avec `MarkdownNodeParser`, tu découpes le document en blocs (\~512–1024 tokens) sur ta machine.                                                                                                                                              | Soit tu fais le chunking localement avant upload, soit tu laisses le service cloud découper pour toi (la plupart des managed RAG pipelines gèrent ça en entrée).                                                   |
| **3. Vectorisation**        | Tu utilises `HuggingFaceEmbedding` (BAAI/bge-large) qui tourne sur ton GPU/CPU local pour transformer chaque chunk en vecteur.                                                                                                               | Tu appelles une API d’**embeddings** (p. ex. OpenAI Embedding API). Tes données (le texte brut du chunk) sont envoyées en HTTPS au cloud, qui te renvoie le vecteur.                                               |
| **4. Stockage vecteurs**    | Tu stockes les vecteurs dans une instance locale de Chroma (Docker ou autre).                                                                                                                                                                | Tu stockes les vecteurs dans un service managé (Pinecone, Weaviate Cloud, Chroma Cloud, Supabase Vector DB…).                                                                                                      |
| **5. Requête utilisateur**  | Tu appelles `query_engine.query(prompt)` qui :<br> • encode localement le prompt en vecteur,<br> • fait la recherche vectorielle dans Chroma local,<br> • monte le contexte,<br> • passe tout ça à Mistral en local pour générer la réponse. | Ton back-end cloud fait :<br> • appel Embedding API sur ton prompt,<br> • recherche dans ton vector store managé,<br> • assemble le contexte,<br> • appel ChatCompletion API (OpenAI, Anthropic…) pour la réponse. |
| **6. Renvoi de la réponse** | Le texte est streamé depuis Mistral sur ta machine jusqu’à ton UI Streamlit.                                                                                                                                                                 | Le texte est streamé depuis l’API LLM cloud jusqu’à ton front-end (ou ton serveur), puis vers ton UI.                                                                                                             
